In [2]:
import pandas as pd
import numpy as np
from six.moves import urllib
import tensorflow as tf
import word2vec
import re
import tempfile

In [5]:
# train word2vec with sample word list dataset
# this might take a minute
vecfile = '/home/claire/Documents/jobs/job-hunting/massively/text8'
word2vec.word2vec(vecfile, '/home/claire/Documents/jobs/job-hunting/massively/text8.bin', size=100, verbose=True)
model = word2vec.load('/home/claire/Documents/jobs/job-hunting/massively/text8.bin')

Starting training using file /home/claire/Documents/jobs/job-hunting/massively/text8


KeyboardInterrupt: 

In [6]:
# load the vector model
model = word2vec.load('/home/claire/Documents/jobs/job-hunting/massively/text8.bin')

In [38]:
# import the sentiment data
df = pd.DataFrame.from_csv('/home/claire/Documents/jobs/job-hunting/massively/sentiment_analysis/dfe_happysad_utf.csv', header=0, sep=',', index_col=0)
df

,features,label
id_nfpu,,
1956967666,Layin n bed with a headache ughhhh...waitin o...,sadness
1956967696,Funeral ceremony...gloomy friday...,sadness
1956968487,"I should be sleep, but im not! thinking about ...",sadness
1956969035,@charviray Charlene my love. I miss you,sadness
1956969172,@kelcouch I'm sorry at least it's Friday?,sadness
1956970047,Ugh! I have to beat this stupid song to get to...,sadness
1956970424,@BrodyJenner if u watch the hills in london u ...,sadness
1956971077,The storm is here and the electricity is gone,sadness
1956971206,So sleepy again and it's not even that late. I...,sadness


In [8]:
# split the data into test and train
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

df_train = df[msk]
df_test = df[~msk]

In [9]:
# split training and testing tweets by sadnesss and happiness
sad_train = df_train.loc[df_train['label'] == 'sadness']
happy_train = df_train.loc[df_train['label'] == 'happiness']

sad_test = df_test.loc[df_test['label'] == 'sadness']
happy_test = df_test.loc[df_test['label'] == 'happiness']

In [10]:
# create numpy arrays
np_sad_train = sad_train.as_matrix(columns=df.columns[0:2])
np_happy_train = happy_train.as_matrix(columns=df.columns[0:2])

np_sad_test = sad_test.as_matrix(columns=df.columns[0:2])
np_happy_test = happy_test.as_matrix(columns=df.columns[0:2])

# or create objects for mean embedding vectorizer

In [39]:
# set maximum sentence length and number of dimension for each word vector
maxSeqLength = df_train['features'].str.len().max()
numFiles = 10362

In [40]:
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def newline(string):
    newline_pattern = re.compile("[^\n]*\n")
    return re.findall(newline_pattern, string)

In [41]:
class indexedArray(np.ndarray):
    def __new__(cls, *args, **kwargs):
        return np.array(*args, **kwargs).view(indexedArray)
    def index(self, value):
        return np.where(self==value)

In [64]:
def vectorize(model, dataFrame):
    ids = np.zeros((numFiles, maxSeqLength), dtype='int64')
    vectors = indexedArray(model.vectors)
    fileCounter = 0
    for tweet in dataFrame:
        print(tweet)
        indexCounter = 0
        cleaned = cleanSentences(tweet[0])
        split = cleaned.split()
        for word in split:
            try:
                index = vectors.index(model[word])
                tuples = [x[0] for x in index]
                ids[fileCounter][indexCounter] = tuples[0]
                print(ids[fileCounter][indexCounter])
            except KeyError:
                ids[fileCounter][indexCounter] = 399999 #Vector for unknown words
                print(ids[fileCounter][indexCounter])
            indexCounter = indexCounter + 1
            if indexCounter >= maxSeqLength:
                break
        fileCounter = fileCounter + 1
        if fileCounter == 4:
            break
    return ids

In [65]:
np.save('idsMatrix', ids)
ids = np.load('idsMatrix.npy')

In [77]:
np.set_printoptions(threshold=np.inf)
ids

KeyboardInterrupt: 

In [67]:
# use scikit-learn to pipeline actual models that will take tokenised text, 
# vectorize and learn to classify the vectors

from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", vectorize(model, np_sad_train)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

['Funeral ceremony...gloomy friday... ' 'sadness']
4131
399999
6460
['@charviray Charlene my love. I miss you ' 'sadness']
399999
399999
609
611
72
5148
207
[ '@BrodyJenner if u watch the hills in london u will realise what tourture it is because were weeks and weeks late  i just watch itonlinelol'
 'sadness']
399999
88
146
3433
1
3112
5
339
146
97
22201
154
399999
27
11
125
40
1997
3
1997
329
72
325
3433
399999
['The storm is here and the electricity is gone ' 'sadness']
1
3828
11
622
3
1
2056
11
3113


TypeError: All intermediate steps should be transformers and implement fit and transform. '[[  4131 399999   6460 ...,      0      0      0]
 [399999 399999    609 ...,      0      0      0]
 [399999     88    146 ...,      0      0      0]
 ..., 
 [     0      0      0 ...,      0      0      0]
 [     0      0      0 ...,      0      0      0]
 [     0      0      0 ...,      0      0      0]]' (type <class 'numpy.ndarray'>) doesn't

In [ ]:
# classify the vectors with a classification model -- SVM, Naive Multinomial Bayesian
# can also do the same thing with weights -- TF-IDF